In [ ]:
from datascience_starter.models.glm import GLM
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:, :2]
y = ((iris.target != 0) * 1)
print(f"X shape is {X.shape}, y shape is {y.shape}")

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)
alpha = model.intercept_
beta = model.coef_
print(f"Alpha: {alpha}, Beta: {beta}")

da = - alpha / beta[0, 1]
dw = - beta[0, 0] / beta[0, 1]
x = np.linspace(4, 8, 100)
db = da + dw * x
sns.scatterplot(X[: , 0], X[: , 1], c=y)
sns.lineplot(x, db, label="decision boundary")

In [ ]:
glm = GLM(likelihood='bernoulli')
glm.fit(X, y, tune=1000)

In [ ]:
print(glm.summary())
glm.plot_trace()

In [ ]:
sns.distplot(glm.trace["beta"][:, 0], label="beta_1")
sns.distplot(glm.trace["beta"][:, 1], label="beta_2")
sns.distplot(glm.trace["alpha"], label="alpha")

In [ ]:
x = np.linspace(4, 8, 100)
sns.scatterplot(X[: , 0], X[: , 1], c=y)
for quant in [0.5]:
    alpha = np.quantile(glm.trace["alpha"], quant, axis=0)
    beta = np.quantile(glm.trace["beta"], quant, axis=0)
    print(f"Alpha: {alpha}, Beta: {beta}")

    da = - alpha / beta[1]
    dw = - beta[0] / beta[1]

    db = da + dw * x
    sns.lineplot(x, db, label=f"decision boundary__{quant}")